In [239]:

# from _utils import make_preds_accumul_aggresive, make_preds_pvalue


from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.pipeline import make_pipeline
from pyriemann.estimation import XdawnCovariances
from pyriemann.tangentspace import TangentSpace
from pyriemann.classification import MDM
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
#from tensorflow import keras
import mne
import os

import sys
import numpy as np
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle
np.set_printoptions(threshold=sys.maxsize)
import matplotlib.pyplot as plt



In [240]:
fps = 60
sfreq = 500


## Path to the data

In [241]:
sfreq = 500

participant = 'P12'
path = '/'.join(['C:\\Users\\s.velut\\Documents\\These\\Protheus_PHD\\Class4\\', participant])
# path = '/home/dcas/k.cabrera/Data/SETNOP2'
n_class=4
fps = 60
window_size = 0.3

# file_name = '_'.join([participant, 'mseq40.set'])
#file_name = '_'.join([participant, 'mseq100.set'])
# file_name = '_'.join([participant, 'burst40.set'])
file_name = '_'.join([participant, 'burst100.set'])
# file_name = '/'.join([path,  participant+'_whitemseq.set'])
# file_name = '_'.join([participant, 'burst', 'oi_1.set'])


#### Load channel positions

## Load the raw data and small pre-process
1. Drop the ACC channels and the shitty channels near ears
2. Average re-referencing
4. Extract 2.2s epochs using events

In [242]:
raw = mne.io.read_raw_eeglab(path+"\\"+file_name, preload=True, verbose=False)

In [243]:
#mne.events_from_annotations(raw)

montage = mne.channels.make_standard_montage("standard_1020")
raw.set_montage(montage)
raw.plot_sensors(show_names=True)
raw.plot_psd()

In [244]:

print(raw.ch_names)
# to_drop = ["P9", "P10", "TP9", "TP10", "10", "21"]
# raw = raw.drop_channels([ch for ch in raw.ch_names if ch in to_drop])
# # raw = raw.drop_channels(["10", "21"])
#keep = ["O1", "O2", "Oz", "P7", "P3", "P4", "P8", "Pz","P9"]
# keep = ["16", "18", "17", "15", "14", "19", "20", "13"] # electrodes to keep
#raw = raw.drop_channels([i for i in raw.ch_names if i not in keep])

# raw = raw.filter(l_freq=50.1, h_freq=49.9, method="iir", verbose=True)
raw = raw.filter(l_freq=1,h_freq=20,  method="fir", verbose=True)
# raw = raw.filter(l_freq=1,h_freq=8,  method="fir", verbose=True)
#raw.resample(250, npad='auto')
# Average re-referencing
mne.set_eeg_reference(raw, 'average', copy=False, verbose=False)

n_channels = len(raw.ch_names)
print("Channels :", n_channels)
print(raw.info)

['Fp1', 'Fz', 'F3', 'F7', 'F9', 'FC5', 'FC1', 'C3', 'T7', 'CP5', 'CP1', 'Pz', 'P3', 'P7', 'P9', 'O1', 'Oz', 'O2', 'P10', 'P8', 'P4', 'CP2', 'CP6', 'T8', 'C4', 'Cz', 'FC2', 'FC6', 'F10', 'F8', 'F4', 'Fp2']
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 20 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 20.00 Hz
- Upper transition bandwidth: 5.00 Hz (-6 dB cutoff frequency: 22.50 Hz)
- Filter length: 1651 samples (3.302 s)

Channels : 32
<Info | 9 non-empty values
 bads: []
 ch_names: Fp1, Fz, F3, F7, F9, FC5, FC1, C3, T7, CP5, CP1, Pz, P3, P7, P9, ...
 chs: 32 EEG
 custom_ref_applied: True
 dig: 32 items (32 EEG)
 highpass: 1.0 Hz
 lowpass: 20.0 Hz
 mea

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


In [245]:
# Strip the annotations that were script to make them easier to process
events, event_id = mne.events_from_annotations(raw, event_id='auto', verbose=False)
to_remove = []
for idx in range(len(raw.annotations.description)):
    if (('collects' in raw.annotations.description[idx]) or
        ('iti' in raw.annotations.description[idx]) or
        (raw.annotations.description[idx] == '[]')):
        to_remove.append(idx)
    else:
        code = raw.annotations.description[idx].split('_')[0]
        lab = raw.annotations.description[idx].split('_')[1]
        code = code.replace('\n', '')
        code = code.replace('[', '')
        code = code.replace(']', '')
        code = code.replace(' ', '')
        raw.annotations.description[idx] = code + '_' + lab

to_remove = np.array(to_remove)
if len(to_remove) > 0:
    raw.annotations.delete(to_remove)
# Get the events
events, event_id = mne.events_from_annotations(raw, event_id='auto', verbose=False)
#print(events.shape)
shift = 0.0
# Epoch the data following event
epochs = mne.Epochs(raw, events, event_id=event_id, tmin=shift, \
            tmax=2.2+shift, baseline=(None, None), preload=False, verbose=False)
labels = epochs.events[..., -1]+1
labels -= np.min(labels)
#print(epochs.events[..., -1])
data = epochs.get_data()
info_ep = epochs.info
#print(epochs)

Using data from preloaded Raw for 60 events and 1101 original time points ...
0 bad epochs dropped


evoke1 = epochs["111111111111110000111100001111000011001111001100001100111111000000110011111100001100110000001111001100000011001100001100000000001100_1"].average()
evoke2 = epochs["000011110000000011111111110000000000001111000011001100110011110011000000110000001111110011001111111111000011000000111100001100111111_2"].average()
evoke3 = epochs["111100000000110000111100000000000011001111001100110011000000111111001111110011001111000000111100111111000000000011111100001100110011_3"].average()
evoke4 = epochs["111100111100111100111100000011111100000011111111110000110011000011110000000011000000001111111111110011001100001111000011000000110011_4"].average()

evoke1.plot(picks="eeg", spatial_colors=True, gfp=True)
# evoke1.plot_topomap()
# evoke1.plot_joint()
evoke2.plot(picks="eeg", spatial_colors=True, gfp=True)
evoke3.plot(picks="eeg", spatial_colors=True, gfp=True)
evoke4.plot(picks="eeg", spatial_colors=True, gfp=True)


### Transform a code in `str` to a code in np.array

In [246]:
def code2array(code):
    tmp = []
    for idx, c in enumerate(code[:-2]):
        if c == '5' or c == '.':
            continue
        elif c == '0':
            if code[idx+2] == '5':
                tmp.append(0.5)
            else:
                tmp.append(0)
        else:
            tmp.append(1)
    if code[-1] == '.':
        if code[-2] == '0':
            tmp.append(0)
        else:
            tmp.append(1)
    return np.array(tmp)

### Build a dictionnary that contains all the code in the np.array format

In [247]:
from collections import OrderedDict
codes = OrderedDict()
for k, v in event_id.items():
    code = k.split('_')[0]
    code = code.replace('.','').replace('2','')
    idx = k.split('_')[1]
    if 'randomslowwhite' in file_name:
        codes[v-1] = code2array(code)
    else:
        codes[v-1] = np.array(list(map(int, code)))

### Define train/test split and windows size
Here we use only the first 7 blocks as calibrition and 8 others would be used as testing

In [248]:
codes

OrderedDict([(0,
              array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                     0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                     0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
                     1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0,
                     0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0,
                     0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0,
                     0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
                     0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
                     0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                     0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                     0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                     0, 0,

In [249]:
print(data.shape)

(60, 32, 1101)


In [250]:
sfreq = int(epochs.info['sfreq'])
n_samples_windows = int(window_size*sfreq)
n_trial_per_class = int(len(data)/n_class)

n_cal = 7

data_train = data[:n_class*n_cal]
labels_train = labels[:n_class*n_cal]
data_test = data[n_class*n_cal:]
labels_test = labels[n_class*n_cal:]


### Slice the epoch in windows
The network is not processing full epochs but windows of 250ms. So each epoch is cut into window and the following code (`0` or `1`) is associated as label.

In [251]:
def to_window_old(data, labels):
    length = int((2.2-window_size)*sfreq)
    X = np.empty(shape=((length)*data.shape[0], n_channels, n_samples_windows))
    y = np.empty(shape=((length)*data.shape[0]), dtype=int)
    print(length)
    print(n_samples_windows)
    count = 0
    for trial_nb, trial in enumerate(data):
        lab = labels[trial_nb]
        c = codes[lab]
        code_pos = 0
        for idx in range(length):
            X[count] = trial[:, idx:idx+n_samples_windows]
            if idx/sfreq >= (code_pos+1)/fps:
                code_pos += 1 
            y[count] = int(c[code_pos])
            count += 1

    # X = np.expand_dims(X, 1)
    X = X.astype(np.float32)
    y_pred = np.vstack((y,np.abs(1-y))).T
    y = np.array([1 if (y >= 0.5) else 0 for y in y_pred[:,0]])
    return X, y

In [252]:
def to_window(data, labels, win_size, data_freq, code_freq, offset=0,
              focus_rising=None, pre_rising=0, post_rising=0,
              focus_falling=None, pre_falling=0, post_falling=0):
    length = int((2.2-win_size)*data_freq)
    X = np.empty(shape=((length)*data.shape[0], n_channels, n_samples_windows))
    Y = np.empty(shape=((length)*data.shape[0]), dtype=int)
    for trial_nb, trial in enumerate(data):
        lab = labels[trial_nb]
        c = codes[lab]
        labels_upsampled = np.repeat(c, sfreq//code_freq)
        labels_upsampled = np.concatenate((np.zeros(int(offset*data_freq), dtype=int), np.array(labels_upsampled)))
        if (focus_rising is not None) or (focus_falling is not None):
            hi_indices = []
            low_indices = []
            for idx in range(1, len(labels_upsampled)):
                if (focus_rising is not None) and (labels_upsampled[idx-1] == 0) and (labels_upsampled[idx] == 1):
                    hi_indices.append(idx)
                elif (focus_falling is not None) and (labels_upsampled[idx-1] == 1) and (labels_upsampled[idx] == 0):
                    low_indices.append(idx)
            focused_labels = np.zeros(length)
            pre_rising_frames = int(sfreq*pre_rising)
            post_rising_frames = int(sfreq*post_rising)
            pre_falling_frames = int(sfreq*pre_falling)
            post_falling_frames = int(sfreq*post_falling)
            for idx in hi_indices:
                focused_labels[idx-pre_rising_frames:idx+post_rising_frames+1] = 1
            for idx in low_indices:
                focused_labels[idx-pre_falling_frames:idx+post_falling_frames+1] = 1
        else:
            focused_labels = labels_upsampled.copy()
            
        for idx in range(length):
            # print('Xidx:', trial_nb*length+idx, "Tidxm:", idx, 'TidxM:', idx +
            #       n_samples_windows, 'Ltrial', trial[:, idx:idx+n_samples_windows].shape)
            X[trial_nb*length+idx] = trial[:, idx:idx+n_samples_windows]
            Y[trial_nb*length+idx] = focused_labels[idx]
    # X = np.expand_dims(X, 1)
    X = X.astype(np.float32)
    y_pred = np.vstack((Y,np.abs(1-Y))).T
    Y = [1 if (Y >= 0.5) else 0 for Y in y_pred[:,0]]
    return X, Y

In [253]:
data_train.shape

(28, 32, 1101)

In [254]:
#window_size = 0.40
# X, Y = to_window_old(np.array(data_train[:1]), labels_train[:1])
# X_train, Y_train = to_window_old(data_train, labels_train)
# X_test, Y_test = to_window_old(data_test, labels_test)
# print(Y[:,0])
X_train, Y_train = to_window_old(data_train, labels_train)#, 0.25, sfreq, 60)
X_test, Y_test = to_window_old(data_test, labels_test)#, 0.25, sfreq, 60)
# print(codes[labels_test[0]])
# print(np.array(Y_train[:,0]))

950
150


950
150


In [255]:
# epochs_train = mne.EpochsArray(X_train,info_ep)
# epochs_train.plot()

In [256]:
# epochs_test = mne.EpochsArray(X_test,info_ep)
# epochs_test.plot()

### Normalization using stats from the train set

In [257]:
X_std = X_train.std(axis=0)
X_train /= X_std + 1e-8
X_std = X_test.std(axis=0)
X_test /= X_std + 1e-8

### Balance classes
Our classes are unbalanced, there are more `1` than `0` in the train set (the stimulation is more often ON than OFF).  
We will use a random under sampler to make it balance.

In [258]:
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(len(Y_train[Y_train[:] == 1]))
print(Y_test.shape)
print(len(Y_test[Y_test[:] == 1]))


(26600, 32, 150)
(30400, 32, 150)
(26600,)
4501
(30400,)
5144


In [259]:

rus = RandomUnderSampler()
counter=np.array(range(0,len(Y_train))).reshape(-1,1)
index,_ = rus.fit_resample(counter,Y_train[:])
X_train = np.squeeze(X_train[index,:,:], axis=1)
Y_train = np.squeeze(Y_train[index])




In [260]:
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)


(9002, 32, 150)
(9002,)
(30400, 32, 150)
(30400,)


In [261]:
print(len(Y_train[Y_train[:] == 0]))
print(len(Y_train[Y_train[:] == 1]))
print(len(Y_test[Y_test[:] == 0]))
print(len(Y_test[Y_test[:] == 1]))

4501
4501
25256
5144


### Pick an architecture

In [262]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline


clf = make_pipeline(XdawnCovariances(nfilter=4, estimator="lwf", xdawn_estimator="lwf"),MDM())
clf2 = make_pipeline(XdawnCovariances(nfilter=4, estimator="lwf", xdawn_estimator="lwf"),
    TangentSpace(),
    LDA(solver="lsqr", shrinkage="auto"))
clf3 = make_pipeline(XdawnCovariances(nfilter=4, estimator="lwf", xdawn_estimator="lwf"),
    TangentSpace(),
    svm.SVC())
clf4 = make_pipeline(XdawnCovariances(estimator='lwf'), TangentSpace(), LogisticRegression(max_iter=10000))


In [272]:
# print(X_train.shape)
# print(Y_train.shape)
# print(X_test.shape)
# print(Y_test.shape)

print(clf3)



Pipeline(steps=[('xdawncovariances',
                 XdawnCovariances(estimator='lwf', xdawn_estimator='lwf')),
                ('tangentspace', TangentSpace()), ('svc', SVC())])


### Cut the train in train and valid
Also set some HP of the network

In [264]:
x_train, x_val, y_train, y_val = train_test_split(X_train, Y_train, test_size=0.2, random_state=42, shuffle=True)


### Attach an optimizer and train the network

res_X_train=[]
for i in range(len(X_train)):
    res_X_train.append(np.array(X_train[i:i+1]).reshape(n_samples_windows, n_channels, 1))
res_X_val=[]
for i in range(len(x_val)):
    res_X_val.append(np.array(x_val[i:i+1]).reshape(n_samples_windows, n_channels, 1))

np.array(res_X_val).shape

np.array(res_X_train).shape

In [265]:
# lr = 1e-3
# weight_decay = 1e-4
# optimizer = keras.optimizers.Adam(learning_rate=lr, amsgrad=True)
# clf.compile(loss='binary_crossentropy',optimizer=optimizer, metrics=['accuracy'])
# X_trainp = xdawn.fit_transform(X_train,y_train)
# X_valp = xdawn.fit_transform(x_val,y_val)
# X_testp = xdawn.fit_trans form(X_test,Y_test)
history = clf.fit(np.array(x_train), y_train)
history2 = clf2.fit(np.array(x_train), y_train)
history3 = clf3.fit(np.array(x_train), y_train)
history4 = clf4.fit(np.array(x_train), y_train)

#keras.backend.clear_session()


In [266]:

print("RG+MDM")
print(history.score(x_train,y_train))
print(history.score(x_val,y_val))
print(history.score(X_test,Y_test))

print("RG+LDA")
print(history2.score(x_train,y_train))
print(history2.score(x_val,y_val))
print(history2.score(X_test,Y_test))

print("RG+SVC")
print(history3.score(x_train,y_train))
print(history3.score(x_val,y_val))
print(history3.score(X_test,Y_test))

print("RG+TS+Reg")
print(history4.score(x_train,y_train))
print(history4.score(x_val,y_val))
print(history4.score(X_test,Y_test))

# print(history.predict(X_trainp)==y_train)



RG+MDM
0.7880849881960839
0.7812326485285952
0.7756907894736842
RG+LDA
0.8273850854048049
0.7945585785674625
0.8033552631578947
RG+SVC
0.977780863768921
0.9711271515824542
0.8185197368421052
RG+TS+Reg
0.8193306485210388
0.8167684619655747
0.7843421052631578


### Model and accuracy and loss
Just check that the model learnt something

filename = ''
np.save(filename + "std_from_calibration", X_std)
model_filename = os.path.join(os.getcwd(), filename + '0.7' + "trainedmodel")
# Save the model if calibration was done
clf.save(model_filename)
keras.backend.clear_session()

## Vizualize learned filters
### Raw vizualiation of 1D convolutinal kernel of the first layer --> spatial filters

### Viz of the corresponding topo maps

### Predict on the test set
The predictions are made on windows to regress the code.  
Here we divide the prediction in 10 fold to avoid OOM from the GPU.

In [267]:

y_pred = clf2.predict(X_test)
y_pred = np.array(y_pred)

In [268]:
y_pred_norm = np.array([1 if (y >= 0.5) else 0 for y in y_pred])
y_test_norm = np.array([0 if y == 0 else 1 for y in Y_test])

In [269]:
import sklearn.metrics
import math
tn, fp, fn, tp = sklearn.metrics.confusion_matrix(y_test_norm, y_pred_norm).ravel()
print("True positives:", tp)
print("True negatives:", tn)
print("False positives:", fp)
print("False negatives:", fn)
print("Accuracy:", (tp+tn)/len(y_test_norm))
print("Sensitivity:", sen:=tp/(tp+fn))
print("Precision:", pre:=tp/(tp+fp))
print("Fowlkes-Mallows:", math.sqrt(sen*pre))


True positives: 2093
True negatives: 22329
False positives: 2927
False negatives: 3051
Accuracy: 0.8033552631578947
Sensitivity: 0.4068818040435459
Precision: 0.41693227091633467
Fowlkes-Mallows: 0.41187638261304893


In [270]:
y_test_norm.shape

(30400,)

### Convert prediction on windows (regressed code) to label prediction

It is offline and synchronous, so we will:
1. First create a `code_buffer` that contains the regressed code on the full epoch (2.2s - the last window)
    - The refresh rate of the EEG device (500 Hz) is faster than the refress rate of the screen (60Hz), so we average predictions (500/60~8 samples) so they correspond to each flip of the screen.
2. Starting from `min_len` (in number of samples), we compute Pearson correalation with the bank of templates code to find the closest one
    - If the most correlated code has a significantely bigger correlation compared to the second one (50% bigger) and the p_value is significative then the trial is classified and we move to the next one
    - If the thresholds are not reached, then we add samples (with a step of 3) to have a longer trial and re-do the computation
    - The thresholds can never been reached in 2.2s, then the trial is not classified.

### Compute accuracy score and accuracy score when a prediction is made (discard not classified trials)

### Other classification method
Same as before but the classification method is different. Instead of thresholds to reach, if when increasing trial lengt a code correllated the most 40 times in a row then the trial is labeled.

In [271]:

acc_best_score=0
acc_best_ratio=0
acc_best_acc=0
acc_best_value = 0
# for minlen in np.linspace(10, 60, 10):
for cons in [15, 20, 25, 30, 35, 40, 45, 50, 55, 60]:
    print(cons)
    labels_pred_accumul, _, mean_long_accumul = make_preds_accumul_aggresive(
        y_pred_norm, codes, min_len=30, sfreq=sfreq, consecutive=cons, window_size=window_size
    )
    print(labels_pred_accumul[labels_pred_accumul != -1])
    print(len(y_pred_norm))
    ratio = np.round(len(labels_pred_accumul[labels_pred_accumul != -1])/len(labels_pred_accumul), 2)
    accuracy = np.round(accuracy_score(labels_test[labels_pred_accumul!=-1], labels_pred_accumul[labels_pred_accumul!=-1]), 2)
    score = np.round(ratio*accuracy,2)
    print("==========", cons, ':', ratio, '--', accuracy,
        '--', score, '--', np.mean(mean_long_accumul))
    if accuracy > acc_best_acc:
        acc_best_score = score
        acc_best_value = cons
        acc_best_acc = accuracy
        acc_best_ratio = ratio

print("Best accuracy", acc_best_acc)
print("Best ratio", acc_best_ratio)
print("Best score", acc_best_score)
print("Best values", acc_best_value)


15


NameError: name 'make_preds_accumul_aggresive' is not defined

In [ ]:
accuracy_score(labels_test[labels_pred_accumul!=-1], labels_pred_accumul[labels_pred_accumul!=-1])

0.71875

In [ ]:
labels_pred_accumul, aaaaa, mean_long_accumul = make_preds_accumul_aggresive(
        y_pred_norm, codes, min_len=30, sfreq=sfreq, consecutive=15, window_size=window_size
    )

In [ ]:
#len(labels_pred_accumul)


In [ ]:
#codes